# Using NumPy arrays to display medical image data

This notebook uses numpy arrays and ipython widgets to interact with medical imagery acquired by an [MRI](https://en.wikipedia.org/wiki/Magnetic_resonance_imaging) scanner. A collection of MRI images (or slices) are stacked together in a data strcuture to form a volume.

### Representing Volumetric Data

Volumetric data can be stored as a 3 dimensional NumPy array. Where a 2D screen element is called a pixel, 
a 3D volume element is called a voxel.

<img src="images/voldata.png" alt="voldata" title="Volumetric Data" width="256" height="256" align="left"/>

### Numpy slicing

Begin with a simple 3D array with a shape of 2 x 2 x 2 for a total of 8 voxels. Another way to think of this is a stack of 2 images (2x2 pixels each) .

In [ ]:
import numpy as np

a = np.array(range(8)).reshape(2,2,2)
a

Use numpy slicing to obtain the first 2x2 slice in the XY plane. The advantage of numpy slicing is that no loops are required to traverse the data set.

In [ ]:
a[0, :, :]

There is a shortcut when working with the XY plane. Just supply the slice index. Use it to obtain the 2nd 2x2 slice in the XY plane.

In [ ]:
a[1]

Obtain the first 2x2 slice in the YZ plane. Note the use of the transpose operator to properly arrange the results.

In [ ]:
a[:, :, 0].T

Obtain the first 2x2 slice in the XZ plane.

In [ ]:
a[:, 0, :]

##### Now its time to get fancy with some real data. First, import some addtional python modules.

In [ ]:
from ipywidgets import interact, interactive, fixed
from skimage import data, filters, io, img_as_float
from IPython.display import Image
from io import BytesIO
import matplotlib as mpl
import requests
import zipfile
import imageio

### Create the volume and display the imagery.

##### Download a collection of MRI images in DICOM format. 

In [ ]:
# 
# The MRI Datasets are taken from https://www.sci.utah.edu/ 
# 
resp = requests.get('http://people.redhat.com/bkozdemb/files/utah-mri-t2.zip')

dicom_zip = BytesIO(resp.content)

#
# Unzip
#
dicom_data = zipfile.ZipFile(dicom_zip)

#
# Read in the DICOM formatted image data files.
# The python comprehension iterates through images and stacks them as a NumPy array.
#
volume = np.array([imageio.imread(dicom_data.open(fname)) for fname in dicom_data.namelist()])

print(f'Volume shape = {volume.shape}, vmin = {volume.min()}, vmax = {volume.max()}')

##### Define the callback functions and the slider widgets

In [ ]:
#
# Callback function for the interact slider.
#
# Use the shortcut method to display a slice in the XY plane.
# 

def edit_image(slice = volume.shape[0] / 2):
    return mpl.pyplot.imshow(volume[slice], cmap=mpl.cm.hot, vmin=volume.min(), vmax=volume.max())

##### Use the slider to slice through the XY plane.

In [ ]:
interact(edit_image, slice = (0, volume.shape[0] - 1, 1));

In [ ]:
#
# Use numpy slicing along with a transpose to view slices in the YZ plane.
# 

def edit_image_02(slice = volume.shape[0] / 2):
    return mpl.pyplot.imshow(volume[:, :, slice].T, cmap=mpl.cm.twilight_shifted, vmin=volume.min(), vmax=volume.max())

##### Use the slider widget to slice througn the YZ plane. This is cool because the data was acquired in the XY plane.

In [ ]:
interact(edit_image_02, slice = (0, volume.shape[0] - 1, 1));

In [ ]:
#
# Use numpy slicing to view slices in the XZ plane.
#

def edit_image_03(slice = volume.shape[0] / 2):
    return mpl.pyplot.imshow(volume[:, slice, :], cmap=mpl.cm.cubehelix, vmin=volume.min(), vmax=volume.max())

##### Finally, use the slider widget to slice througn the XZ plane.

In [ ]:
interact(edit_image_03, slice = (0, volume.shape[0] - 1, 1));

##### References

[[1] Scientific Computing and Imaging Institute](https://www.sci.utah.edu/)

[[2] Oak Tree Technologies](https://www.oak-tree.tech/)

[[3] Stack overflow discussion](https://stackoverflow.com/questions/53837060/extract-sagittal-and-coronal-cuts-from-axial-view-using-pydicom)